# Stommel Gyre on Unstructured Grid
This tutorial walks through creating a UXArray dataset using the Stommel Gyre analytical solution for a closed rectangular domain on a beta-plane

In [1]:
def stommel_fieldset_uxarray(xdim=200, ydim=200):
    """Simulate a periodic current along a western boundary, with significantly
    larger velocities along the western edge than the rest of the region

    The original test description can be found in: N. Fabbroni, 2009,
    Numerical Simulation of Passive tracers dispersion in the sea,
    Ph.D. dissertation, University of Bologna
    http://amsdottorato.unibo.it/1733/1/Fabbroni_Nicoletta_Tesi.pdf
    """
    import uxarray as ux
    import numpy as np
    import math
    import pandas as pd

    a = b = 66666 * 1e3
    scalefac = 0.00025  # to scale for physically meaningful velocities

    # Coordinates of the test fieldset
    # Crowd points to the west edge of the domain
    # using a polyonmial map on x-direction
    x = np.linspace(0, 1, xdim, dtype=np.float32)
    lon, lat = np.meshgrid(
        a*x,
        np.linspace(0, b, ydim, dtype=np.float32)
    )
    points = (lon.flatten()/1111111.111111111,lat.flatten()/1111111.111111111)
    
    # Create the grid
    uxgrid = ux.Grid.from_points(points, method="regional_delaunay")
    uxgrid.construct_face_centers()

    # Define arrays U (zonal), V (meridional) and P (sea surface height)
    U = np.zeros((1,1,lat.size), dtype=np.float32)
    V = np.zeros((1,1,lat.size), dtype=np.float32)
    P = np.zeros((1,1,lat.size), dtype=np.float32)

    beta = 2e-11
    r = 1 / (11.6 * 86400)
    es = r / (beta * a)

    i = 0
    for x, y in zip(lon.flatten(), lat.flatten()):
        xi = x / a
        yi = y / b
        P[0,0,i] = (
            (1 - math.exp(-xi / es) - xi)
            * math.pi
            * np.sin(math.pi * yi)
            * scalefac
        )
        U[0,0,i] = (
            -(1 - math.exp(-xi / es) - xi)
            * math.pi**2
            * np.cos(math.pi * yi)
            * scalefac
        )
        V[0,0,i] = (
            (math.exp(-xi / es) / es - 1)
            * math.pi
            * np.sin(math.pi * yi)
            * scalefac
        )
        i+=1

    u = ux.UxDataArray(
        data=U,
        name='u',
        uxgrid=uxgrid,
        dims=["time","nz1","n_node"],
        coords = dict(
            time=(["time"], pd.to_datetime(['2000-01-01'])),
            nz1=(["nz1"], [0]),
        ),
        attrs=dict(
            description="zonal velocity",
            units="m/s",
            location="node",
            mesh="delaunay",
        ),
    )
    v = ux.UxDataArray(
        data=V,
        name='v',
        uxgrid=uxgrid,
        dims=["time","nz1","n_node"],
        coords = dict(
            time=(["time"], pd.to_datetime(['2000-01-01'])),
            nz1=(["nz1"], [0]),
        ),
        attrs=dict(
            description="meridional velocity",
            units="m/s",
            location="node",
            mesh="delaunay",
        ),
    )
    p = ux.UxDataArray(
        data=P,
        name='p',
        uxgrid=uxgrid,
        dims=["time","nz1","n_node"],
        coords = dict(
            time=(["time"], pd.to_datetime(['2000-01-01'])),
            nz1=(["nz1"], [0]),
        ),
        attrs=dict(
            description="pressure",
            units="N/m^2",
            location="node",
            mesh="delaunay",
        ),
    )


    return ux.UxDataset(
        {'u':u, 'v':v, 'p': p}, 
        uxgrid=uxgrid
    )

uxds = stommel_fieldset_uxarray(50,50)

uxds.uxgrid.plot(
    line_width=0.5,
    height=500,
    width=1000,
    title="Regional Delaunay Regions",
)



/home/joe/miniconda3/envs/parcels/lib/python3.13/site-packages/uxarray/grid/coordinates.py:255: UserWarning: This cannot be guaranteed to work correctly on concave polygons
  warnings.warn("This cannot be guaranteed to work correctly on concave polygons")


:Path   [Longitude,Latitude]

In [10]:
def stommel_fieldset_xarray(xdim=200, ydim=200, grid_type="A"):
    """Simulate a periodic current along a western boundary, with significantly
    larger velocities along the western edge than the rest of the region

    The original test description can be found in: N. Fabbroni, 2009,
    Numerical Simulation of Passive tracers dispersion in the sea,
    Ph.D. dissertation, University of Bologna
    http://amsdottorato.unibo.it/1733/1/Fabbroni_Nicoletta_Tesi.pdf
    """
    import xarray as xr
    import numpy as np
    import math
    import pandas as pd

    a = b = 10000 * 1e3
    scalefac = 0.05  # to scale for physically meaningful velocities
    dx, dy = a / xdim, b / ydim

    # Coordinates of the test fieldset (on A-grid in deg)
    lon = np.linspace(0, a, xdim, dtype=np.float32)
    lat = np.linspace(0, b, ydim, dtype=np.float32)

    # Define arrays U (zonal), V (meridional) and P (sea surface height)
    U = np.zeros((1,1,lat.size, lon.size), dtype=np.float32)
    V = np.zeros((1,1,lat.size, lon.size), dtype=np.float32)
    P = np.zeros((1,1,lat.size, lon.size), dtype=np.float32)

    beta = 2e-11
    r = 1 / (11.6 * 86400)
    es = r / (beta * a)

    for j in range(lat.size):
        for i in range(lon.size):
            xi = lon[i] / a
            yi = lat[j] / b
            P[...,j, i] = (
                (1 - math.exp(-xi / es) - xi)
                * math.pi
                * np.sin(math.pi * yi)
                * scalefac
            )
            if grid_type == "A":
                U[...,j, i] = (
                    -(1 - math.exp(-xi / es) - xi)
                    * math.pi**2
                    * np.cos(math.pi * yi)
                    * scalefac
                )
                V[...,j, i] = (
                    (math.exp(-xi / es) / es - 1)
                    * math.pi
                    * np.sin(math.pi * yi)
                    * scalefac
                )

    time =  pd.to_datetime(['2000-01-01'])
    z = [0]
    if grid_type == "C":
        V[...,:, 1:] = (P[...,:, 1:] - P[...,:, 0:-1]) / dx * a
        U[...,1:, :] = -(P[...,1:, :] - P[...,0:-1, :]) / dy * b
        u_dims = ["time","nz1","face_lat", "node_lon"]
        u_lat = lat
        u_lon = lon - dx * 0.5
        u_location = "x_edge"
        v_dims = ["time","nz1","node_lat", "face_lon"]
        v_lat = lat - dy * 0.5
        v_lon = lon
        v_location = "y_edge"
        p_dims = ["time","nz1","face_lat", "face_lon"]
        p_lat = lat
        p_lon = lon
        p_location = "face"
        
    else:
        u_dims = ["time","nz1","node_lat", "node_lon"]
        v_dims = ["time","nz1","node_lat", "node_lon"]
        p_dims = ["time","nz1","node_lat", "node_lon"]
        u_lat = lat
        u_lon = lon
        v_lat = lat
        v_lon = lon
        u_location = "node"
        v_location = "node"
        p_lat = lat
        p_lon = lon
        p_location = "node"

    u = xr.DataArray(
        data=U,
        name='u',
        dims=u_dims,
        coords = [time,z,u_lat,u_lon],
        attrs=dict(
            description="zonal velocity",
            units="m/s",
            location=u_location,
            mesh=f"Arakawa-{grid_type}",
        ),
    )
    v = xr.DataArray(
        data=V,
        name='v',
        dims=v_dims,
        coords = [time,z,v_lat,v_lon],
        attrs=dict(
            description="meridional velocity",
            units="m/s",
            location=v_location,
            mesh=f"Arakawa-{grid_type}",
        ),
    )
    p = xr.DataArray(
        data=P,
        name='p',
        dims=p_dims,
        coords = [time,z,p_lat,p_lon],
        attrs=dict(
            description="pressure",
            units="N/m^2",
            location=p_location,
            mesh=f"Arakawa-{grid_type}",
        ),
    )

    return xr.Dataset(
        {'u':u, 'v':v, 'p': p}
    )

ds_arakawa_a = stommel_fieldset_xarray(50,50,"A")
ds_arakawa_c = stommel_fieldset_xarray(50,50,"C")


In [11]:
ds_arakawa_a

<xarray.Dataset> Size: 30kB
Dimensions:   (time: 1, nz1: 1, node_lat: 50, node_lon: 50)
Coordinates:
  * time      (time) datetime64[ns] 8B 2000-01-01
  * nz1       (nz1) int64 8B 0
  * node_lat  (node_lat) float32 200B 0.0 2.041e+05 ... 9.796e+06 1e+07
  * node_lon  (node_lon) float32 200B 0.0 2.041e+05 ... 9.796e+06 1e+07
Data variables:
    u         (time, nz1, node_lat, node_lon) float32 10kB -0.0 -0.4752 ... 0.0
    v         (time, nz1, node_lat, node_lon) float32 10kB 0.0 0.0 ... 1.373e-08
    p         (time, nz1, node_lat, node_lon) float32 10kB 0.0 0.0 ... -0.0

In [12]:
ds_arakawa_a["u"].attrs

{'description': 'zonal velocity',
 'units': 'm/s',
 'location': 'node',
 'mesh': 'Arakawa-A'}

In [13]:
ds_arakawa_c

<xarray.Dataset> Size: 31kB
Dimensions:   (time: 1, nz1: 1, face_lat: 50, node_lon: 50, node_lat: 50,
               face_lon: 50)
Coordinates:
  * time      (time) datetime64[ns] 8B 2000-01-01
  * nz1       (nz1) int64 8B 0
  * face_lat  (face_lat) float32 200B 0.0 2.041e+05 ... 9.796e+06 1e+07
  * node_lon  (node_lon) float32 200B -1e+05 1.041e+05 ... 9.696e+06 9.9e+06
  * node_lat  (node_lat) float32 200B -1e+05 1.041e+05 ... 9.696e+06 9.9e+06
  * face_lon  (face_lon) float32 200B 0.0 2.041e+05 ... 9.796e+06 1e+07
Data variables:
    u         (time, nz1, face_lat, node_lon) float32 10kB 0.0 0.0 ... 0.0
    v         (time, nz1, node_lat, face_lon) float32 10kB 0.0 0.0 ... 1.401e-08
    p         (time, nz1, face_lat, face_lon) float32 10kB 0.0 0.0 ... -0.0

In [14]:
import numpy as np
min_length_scale = 1111111.111111111*np.sqrt(np.min(uxds.uxgrid.face_areas))
print(min_length_scale)

max_v = np.sqrt(uxds['u']**2 + uxds['v']**2).max()
print(max_v)

cfl = 0.1
dt = cfl * min_length_scale / max_v
print(dt)

<xarray.DataArray 'face_areas' ()> Size: 8B
array(1142.17017624)
<xarray.DataArray ()> Size: 8B
array(1.04820871)
<xarray.DataArray ()> Size: 8B
array(108.96400321)


In [15]:
import uxarray as ux
from datetime import timedelta
from parcels import (
    UXFieldSet,
    ParticleSet,
    Particle,
    UxAdvectionEuler
)
import numpy as np

npart = 10
fieldset = UXFieldSet(uxds)
# pset = ParticleSet(
#         fieldset, 
#         pclass=Particle, 
#         lon=np.linspace(1, 59, npart), 
#         lat=np.zeros(npart)+30)
# pset.execute(UxAdvectionEuler, runtime=timedelta(hours=24), dt=timedelta(seconds=dt))


: 